In [43]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from sklearn.model_selection import KFold, cross_val_score
from sklearn.tree import DecisionTreeClassifier

In [44]:
data = pd.read_csv('loan_data.csv')
df = pd.DataFrame(data)

In [45]:
df.isnull().sum()

person_age                        0
person_gender                     0
person_education                  0
person_income                     0
person_emp_exp                    0
person_home_ownership             0
loan_amnt                         0
loan_intent                       0
loan_int_rate                     0
loan_percent_income               0
cb_person_cred_hist_length        0
credit_score                      0
previous_loan_defaults_on_file    0
loan_status                       0
dtype: int64

In [46]:
df.describe(include= 'all')

,person_age,person_gender,person_education,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_intent,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file,loan_status
count,45000.000000,45000,45000,4.500000e+04,45000.000000,45000,45000.000000,45000,45000.000000,45000.000000,45000.000000,45000.000000,45000,45000.000000
unique,NaN,2,5,NaN,NaN,4,NaN,6,NaN,NaN,NaN,NaN,2,NaN
top,NaN,male,Bachelor,NaN,NaN,RENT,NaN,EDUCATION,NaN,NaN,NaN,NaN,Yes,NaN
freq,NaN,24841,13399,NaN,NaN,23443,NaN,9153,NaN,NaN,NaN,NaN,22858,NaN
mean,27.764178,NaN,NaN,8.031905e+04,5.410333,NaN,9583.157556,NaN,11.006606,0.139725,5.867489,632.608756,NaN,0.222222
std,6.045108,NaN,NaN,8.042250e+04,6.063532,NaN,6314.886691,NaN,2.978808,0.087212,3.879702,50.435865,NaN,0.415744
min,20.000000,NaN,NaN,8.000000e+03,0.000000,NaN,500.000000,NaN,5.420000,0.000000,2.000000,390.000000,NaN,0.000000
25%,24.000000,NaN,NaN,4.720400e+04,1.000000,NaN,5000.000000,NaN,8.590000,0.070000,3.000000,601.000000,NaN,0.000000
50%,26.000000,NaN,NaN,6.704800e+04,4.000000,NaN,8000.000000,NaN,11.010000,0.120000,4.000000,640.000000,NaN,0.000000
75%,30.000000,NaN,NaN,9.578925e+04,8.000000,NaN,12237.250000,NaN,12.990000,0.190000,8.000000,670.000000,NaN,0.000000


In [47]:
def encoding(col):
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

In [48]:
# removing null values with the mode (most repeated value)
def remove_outlier(col):
    global df
    
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1

    upper_limit = q3 + (1.5 * iqr)
    lower_limit = q1 - (1.5 * iqr)

    df = df.loc[(df[col] >= lower_limit) & (df[col] <= upper_limit) ]

In [49]:
#incode to person_gender, person_education, home_ownership, loan_intent, pervious_loan_defaults
# outlires age, income, emp_exp, loan_amount, int_rate, cred_hist_length, credit_score,

cols = ['person_gender', 'person_home_ownership', 'person_education', 'loan_intent',
       'previous_loan_defaults_on_file']
outliers = ['person_age', 'person_income',
       'person_emp_exp', 'loan_amnt', 'loan_int_rate', 'cb_person_cred_hist_length', 'credit_score']

# removing outliers from specific list
for col in outliers:
    remove_outlier(col)

# encoding cols in specific list
for col in cols:
    encoding(col)
    
# df['loan_status']

In [50]:
df.dropna()
df.drop_duplicates()


,person_age,person_gender,person_education,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_intent,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file,loan_status
1,21.0,0,3,12282.0,0,2,1000.0,1,11.14,0.08,2.0,504,1,0
2,25.0,0,3,12438.0,3,0,5500.0,3,12.87,0.44,3.0,635,0,1
5,21.0,0,3,12951.0,0,2,2500.0,5,7.14,0.19,2.0,532,0,1
9,21.0,0,3,12739.0,0,2,1600.0,5,14.74,0.13,3.0,640,0,1
11,21.0,0,0,13113.0,0,2,4500.0,2,8.63,0.34,2.0,651,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44994,24.0,0,0,31924.0,2,3,12229.0,3,10.70,0.38,4.0,678,0,1
44995,27.0,1,0,47971.0,6,3,15000.0,3,15.66,0.31,3.0,645,0,1
44997,33.0,1,0,56942.0,7,3,2771.0,0,10.02,0.05,10.0,668,0,1
44998,29.0,1,1,33164.0,4,3,12000.0,1,13.23,0.36,6.0,604,0,1


In [51]:
x = df.drop(columns=['loan_status'])
y = df['loan_status']


In [52]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score

In [53]:
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.2, random_state= 12)

# lasso_model = Ridge(alpha=0.5)
# lasso_model.fit(x_train, y_train)

# y_pred = lasso_model.predict(x_test) > 0.5

# mse = mean_squared_error(y_test, y_pred)

# print(f"Mean square error: {mse}")


In [54]:
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
import matplotlib.pyplot as plt

In [55]:
rfe = RFE(estimator= DecisionTreeClassifier(), n_features_to_select= 4)
rfe.fit(x, y)

RFE(estimator=DecisionTreeClassifier(), n_features_to_select=4)

In [56]:
for i, col in zip(range(x.shape[1]), x.columns):
    print(f'{col} selected: {rfe.support_[i]}, rank = {rfe.ranking_[i]}')

person_age selected: False, rank = 8
person_gender selected: False, rank = 10
person_education selected: False, rank = 9
person_income selected: True, rank = 1
person_emp_exp selected: False, rank = 7
person_home_ownership selected: False, rank = 4
loan_amnt selected: False, rank = 3
loan_intent selected: False, rank = 5
loan_int_rate selected: True, rank = 1
loan_percent_income selected: True, rank = 1
cb_person_cred_hist_length selected: False, rank = 6
credit_score selected: False, rank = 2
previous_loan_defaults_on_file selected: True, rank = 1


In [57]:
features=  x.columns
ranks = rfe.ranking_
selected = rfe.support_

rfe_results = pd.DataFrame({
    'feature' : features,
    'ranks' : ranks,
    'selected' : selected
})
rfe_results.sort_values(by = 'ranks', inplace= True)

In [58]:
rfe_results

,feature,ranks,selected
3,person_income,1,True
12,previous_loan_defaults_on_file,1,True
9,loan_percent_income,1,True
8,loan_int_rate,1,True
11,credit_score,2,False
6,loan_amnt,3,False
5,person_home_ownership,4,False
7,loan_intent,5,False
10,cb_person_cred_hist_length,6,False
4,person_emp_exp,7,False


In [59]:
df = df[['person_income', 'previous_loan_defaults_on_file', 'loan_percent_income', 'loan_int_rate', 'loan_status']]
df.loc[df['loan_status'] == 0]

,person_income,previous_loan_defaults_on_file,loan_percent_income,loan_int_rate,loan_status
1,12282.0,1,0.08,11.14,0
23,13866.0,1,0.11,7.29,0
27,14293.0,0,0.10,9.32,0
59,15229.0,1,0.11,11.01,0
65,15013.0,0,0.10,14.84,0
...,...,...,...,...,...
42102,36900.0,1,0.05,5.77,0
42103,60006.0,1,0.05,8.92,0
42104,64146.0,1,0.11,10.28,0
42106,110291.0,1,0.12,13.29,0


In [60]:
from sklearn.metrics import classification_report

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.2, random_state= 12)

x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5, random_state= 12)


# dtc = DecisionTreeClassifier(random_state= 12)


# y_pred = dtc.predict(x_test)

# accuracy = accuracy_score(y_test, y_pred)
# precision = precision_score(y_test, y_pred, average= 'weighted')
# recall = recall_score(y_test, y_pred, average= 'weighted')
# f1_score = f1_score(y_test, y_pred, average= 'weighted')




In [61]:
def apply_model(model):
    model.fit(x_train, y_train)
    test_pred = model.predict(x_test)
    train_pred = model.predict(x_train)
    print('Train Classification Report: ')
    print(classification_report(y_train, train_pred))
    print('Test Classification Report: ')
    print(classification_report(y_test, test_pred))




In [62]:
dtc = DecisionTreeClassifier(random_state= 12)
apply_model(dtc)

Train Classification Report: 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     23248
           1       1.00      1.00      1.00      6565

    accuracy                           1.00     29813
   macro avg       1.00      1.00      1.00     29813
weighted avg       1.00      1.00      1.00     29813

Test Classification Report: 
              precision    recall  f1-score   support

           0       0.93      0.94      0.93      2903
           1       0.78      0.75      0.76       824

    accuracy                           0.90      3727
   macro avg       0.85      0.84      0.85      3727
weighted avg       0.90      0.90      0.90      3727

